In [1]:
from gensim.models import Word2Vec

# Data

In [2]:
import json
import requests

def load_data_skill(filename):

    # resp = requests.get(filename)
    with open(filename, 'r') as f:
        data = json.load(f)

    return list(data)

# filename = 'data/train/total_skill_augmentation_jds_sorted_relevant_combine_random_and_not.json'
filename = 'data/train/total_skill_jds_augmentation.json'
total_skills_jds = load_data_skill(filename)
len(total_skills_jds)

175971

# Valid Function

In [3]:
def model_prediction(model, skills, topn=20):
    result = {}
    for skill in skills:
        try:
            topn_skills = [skill[0] for skill in model.wv.most_similar(skill, topn=topn)]
        except:
            continue

        result[skill] = topn_skills

    return result

def valid_skill(prediction, valid_data): # prediction: dict, valid_data: dict
    count = 0
    for skill in prediction.keys():
        for rlv_skill in prediction[skill]:
            if rlv_skill in valid_data[skill]:
                count += 1

    flattened_valid = []
    for skills in prediction.keys():
        flattened_valid += valid_data[skills]
        
    # find the number of skill in valid data
    total_skills = len(flattened_valid)

    return count/total_skills


# Model training

In [4]:
# valid data
valid_data_path = 'data/valid/skills_valid.json'

with open(valid_data_path, 'r') as f:
    valid_data = json.load(f)

f.close()

# valid_data = json.loads(resp.text)

# total valid skill
total_valid_skill = 0
for skill in valid_data.keys():
    total_valid_skill += len(valid_data[skill])

print(f'Total valid skill: {total_valid_skill}')

Total valid skill: 151578


In [5]:
import random

def calculate_acc(model):
  #pick random 200 skills in valid data

  prediction = model_prediction(model, valid_data.keys())

  accuracy = valid_skill(prediction, valid_data)


  return accuracy

# calculate_acc(model=model)
# print(f'Accuracy: {sum(accuracies)/100}')

In [6]:
import itertools

# create hyperparameter for training fasttext model and use grid search to find best hyperparameter
# Define hyperparameters
vector_size = [100]  # Can adjust this to your desired vector size
window_size = [5]    # Context window size
min_count = [12]      # Minimum word frequency
sg = [1]             # Skip-gram model (1 for skip-gram, 0 for CBOW)
hs = [0]                # Using negative sampling (1 for HS, 0 for negative sampling)
negative = [2, 3, 5, 10]
ns_exponent = [0.5, 0.75, 1]
seed = [1]
batch_word = [100]     # Number of words for each training batch
shrink_windows = [False, True] # Whether to shrink the window size as training goes on. This usually improves the accuracy of word vectors
alpha = [0.025]           # The initial learning rate

workers = [2]           # Number of worker threads to train the model
epochs = [200]          # Number of epochs to train the model

# Create combinations of hyperparameters
hyperparameter_combinations = list(itertools.product(vector_size, window_size, min_count, sg, hs, negative, ns_exponent, seed, batch_word, shrink_windows, alpha, workers, epochs))

# Print the total number of combinations
print(f"Total number of hyperparameter combinations: {len(hyperparameter_combinations)}")

Total number of hyperparameter combinations: 24


In [7]:
def calculate_acc(model):
  
  prediction = model_prediction(model, valid_data.keys())

  accuracy = valid_skill(prediction, valid_data)


  return accuracy

In [8]:
# function train model fasttext
def train_word2vec(skills_jds,
                   vector_size=100,
                   window_size=5,
                   min_count=3,
                   sg=1,
                   hs=1,
                   negative=0,
                   ns_exponent=0,
                   seed=1,
                   batch_word=100,
                   shrink_windows=True,
                   alpha=0.025,
                   workers=2,
                   epochs=100):

    # Initialize and train the FastText model
    model = Word2Vec(
        vector_size=vector_size,
        window=window_size,
        min_count=min_count,
        sg=sg,
        hs=hs,
        negative=negative,
        alpha=alpha,
        ns_exponent=ns_exponent,
        seed=seed,
        batch_words=batch_word,
        shrink_windows=shrink_windows,
        workers=workers,
        epochs=2
    )

    # Build the vocabulary
    model.build_vocab(corpus_iterable=skills_jds)

    model.train(
        corpus_iterable=skills_jds,
        total_words = model.corpus_total_words,
        total_examples=len(skills_jds),
        epochs=model.epochs
    )

    pre_accuaracy = calculate_acc(model)
    print(f'Epoch {1}/{epochs} - Accuaracy: {pre_accuaracy}')

    # threshold = 0.001
    num_epochs = epochs - 1
    break_count = 3


    for epoch in range(num_epochs):

        if break_count <= 0:
            break
        model.update_weights()

        model.train(
            corpus_iterable=skills_jds,
            total_words = model.corpus_total_words,
            total_examples=len(skills_jds),
            epochs=model.epochs
        )

        accuaracy = calculate_acc(model)

        print(f'Epoch {epoch+2}/{epochs} - Accuaracy: {accuaracy}')

        # early stopping
        if pre_accuaracy - accuaracy > 0:
            break_count -= 1
        
        pre_accuaracy = accuaracy
        
    return model

In [22]:
import multiprocessing
     
# Define hyperparameters
vector_size = 100  # Can adjust this to your desired vector size
window_size = 5    # Context window size
min_count = 6      # Minimum word frequency
sg = 1             # Skip-gram model (1 for skip-gram, 0 for CBOW)
hs = 1                # Using negative sampling (1 for HS, 0 for negative sampling)
negative = 0
ns_exponent = 0    # The exponent used to shape the negative sampling distribution (0 for unigram, 1 for uniform)
batch_word = 100     # Number of words for each training batch
shrink_windows = True # Whether to shrink the window size as training goes on. This usually improves the accuracy of word vectors
alpha = 0.01           # The initial learning rate
workers=2
seed=1

epochs = 200          # Number of epochs to train the model

In [23]:
model = train_word2vec(skills_jds=total_skills_jds,
                       vector_size=vector_size,
                       window_size=window_size,
                       min_count=min_count,
                       workers=workers,
                       alpha=alpha,
                       sg=sg,
                       hs=hs,
                       negative=negative,
                       ns_exponent=ns_exponent,
                       batch_word=batch_word,
                       shrink_windows=shrink_windows,
                       epochs=epochs)

In [24]:
accuracy = calculate_acc(model)
accuracy

0.10548142805625677

In [ ]:
import os

model_path = 'word2vec_simple'
idx = 1

log_parameters = {
            "vector_size": vector_size,
            "windows_size": window_size,
            "min_cout": min_count,
            "sg": sg,
            "hs": hs,
            "negative": negative,
            "ns_exponent": ns_exponent,
            "seed": seed,
            "batch_word": batch_word,
            "shrink_windows": shrink_windows,
            "alpha": alpha,
            "workers": workers,
            "epochs": epochs
        }

if not os.path.exists(f'{model_path}/{idx}'):
    os.makedirs(f'{model_path}/{idx}')

model.save(f'{model_path}/{idx}/word2vec.model')

# export model
vocab = model.wv.key_to_index

with open(f'{model_path}/{idx}/vocab.tsv', "w", encoding='utf-8') as f:
    # f.write("word\n")
    for word in vocab:
        f.write(f"{word}\n")

f.close()

with open(f'{model_path}/{idx}/vectors.tsv', "w", encoding='utf-8') as f:
    for word in vocab:
        vector = "\t".join(str(val) for val in model.wv[word])

        f.write(f"{vector}\n")
    f.close()

with open(f'{model_path}/{idx}/logmodel.txt', 'w', encoding='utf-8') as f:
    f.write(str(log_parameters) + '\n')
    f.write(str(accuracy))
f.close()

## Grid search

In [9]:
import os
import numpy as np

def grid_search(model_path = 'word2vecmodel', total_skill_jds=total_skills_jds):
    word2vec_models = []
    accuracy_models = []
    for idx, parameters in enumerate(hyperparameter_combinations):

        log_parameters = {
            "vector_size": parameters[0],
            "windows_size": parameters[1],
            "min_cout": parameters[2],
            "sg": parameters[3],
            "hs": parameters[4],
            "negative": parameters[5],
            "ns_exponent": parameters[6],
            "seed": parameters[7],
            "batch_words": parameters[8],
            "shrink_windows": parameters[9],
            "alpha": parameters[10],
            "workers": parameters[11],
            "epochs": parameters[12]
        }
        print(f"Training model with parameters: {log_parameters}")

        model = train_word2vec(total_skill_jds, parameters[0], parameters[1], parameters[2],
                    parameters[3], parameters[4], parameters[5],
                    parameters[6], parameters[7], parameters[8],
                    parameters[9], parameters[10], parameters[11], parameters[12])
        # fasttext_models.append(model)

        # valid model
        prediction = model_prediction(model, valid_data.keys())
        word2vec_models.append(model)

        accuracy = valid_skill(prediction, valid_data)
        accuracy_models.append(accuracy)

        print(f'Accuracy: {accuracy}')

        # save model
        if not os.path.exists(f'{model_path}/{idx}'):
            os.makedirs(f'{model_path}/{idx}')

        model.save(f'{model_path}/{idx}/word2vec.model')

        # export model
        vocab = model.wv.key_to_index

        with open(f'{model_path}/{idx}/vocab.tsv', "w", encoding='utf-8') as f:
            # f.write("word\n")
            for word in vocab:
                f.write(f"{word}\n")

        f.close()

        with open(f'{model_path}/{idx}/vectors.tsv', "w", encoding='utf-8') as f:
            for word in vocab:
                vector = "\t".join(str(val) for val in model.wv[word])

                f.write(f"{vector}\n")
        f.close()

        with open(f'{model_path}/{idx}/logmodel.txt', 'w', encoding='utf-8') as f:
            f.write(str(log_parameters) + '\n')
            f.write(str(accuracy))
        f.close()


    # #find best model
    # best_model = fasttext_models[np.argmax(accuracy_models)]
    # best_accuracy = max(accuracy_models)

    # # save best model
    # if not os.path.exists(f'{model_path}/best_model'):
    #     os.makedirs(f'{model_path}/best_model')

    # best_model.save(f'{model_path}/best_model/fasttext_model.model')
    # # save best parameter
    # with open(f'{model_path}/best_model/best_parameter.txt', 'w') as f:
    #     f.write(f'Best parameter: {hyperparameter_combinations[np.argmax(accuracy_models)]}\n')
    #     f.write(f'Best accuracy: {best_accuracy}')

    return word2vec_models, accuracy_models

In [10]:
models = grid_search('word2vecmodel_hs_1_group_model_1', total_skills_jds)

Training model with parameters: {'vector_size': 100, 'windows_size': 5, 'min_cout': 12, 'sg': 1, 'hs': 0, 'negative': 2, 'ns_exponent': 0.5, 'seed': 1, 'batch_words': 100, 'shrink_windows': False, 'alpha': 0.025, 'workers': 2, 'epochs': 200}
Epoch 1/200 - Accuaracy: 0.10364431780763002
Epoch 2/200 - Accuaracy: 0.12359633619386184
Epoch 3/200 - Accuaracy: 0.13365634681164193
Epoch 4/200 - Accuaracy: 0.14123663537333594
Epoch 5/200 - Accuaracy: 0.1472376971513437
Epoch 6/200 - Accuaracy: 0.15197537750240245
Epoch 7/200 - Accuaracy: 0.15561767927584053
Epoch 8/200 - Accuaracy: 0.1587694127292399
Epoch 9/200 - Accuaracy: 0.16144401809054681
Epoch 10/200 - Accuaracy: 0.1635205333082449
Epoch 11/200 - Accuaracy: 0.16501911872425357
Epoch 12/200 - Accuaracy: 0.16625561969531003
Epoch 13/200 - Accuaracy: 0.16764668328774857
Epoch 14/200 - Accuaracy: 0.1687151814094767
Epoch 15/200 - Accuaracy: 0.16957535599803772
Epoch 16/200 - Accuaracy: 0.17038848978878682
Epoch 17/200 - Accuaracy: 0.1710672

KeyboardInterrupt: 

# Test model

In [6]:
vector = model.wv['html']  # get numpy vector of a word
sims = model.wv.most_similar('hive', topn=10)  # get other similar words
sims

[('sap', 0.686120867729187),
 ('plant construction', 0.680181622505188),
 ('genomics', 0.6622642278671265),
 ('snowflake', 0.6429843306541443),
 ('scrum', 0.6354222297668457),
 ('obstetrics', 0.6297516822814941),
 ('mathematics', 0.6162903308868408),
 ('business intelligence', 0.5987536311149597),
 ('splunk', 0.5981040596961975),
 ('olap', 0.5967950820922852)]

# End